In [1]:
pip install mord

Note: you may need to restart the kernel to use updated packages.


In [8]:
%matplotlib inline
from pathlib import Path

import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from mord import LogisticIT
import matplotlib.pylab as plt
import seaborn as sns
from dmba import classificationSummary, gainsChart, liftChart
from dmba.metric import AIC_score

In [9]:
imdb_df = pd.read_csv('projectData.csv')
imdb_df = imdb_df.set_index('tconst')

imdb_df = imdb_df.drop('numVotes', axis=1)
imdb_df = imdb_df.drop('runtimeMinutes', axis=1)
imdb_df = imdb_df.drop('genres', axis=1)
imdb_df = imdb_df.drop('directors', axis=1)
imdb_df = imdb_df.drop('writers', axis=1)
# imdb_df = imdb_df.drop('actor', axis=1)

In [10]:
# genres_df = imdb_df['genres'].str.get_dummies(sep=',')
# genres_df = genres_df.add_prefix('genre_')
# imdb_df = pd.concat([imdb_df, genres_df], axis=1)
# imdb_df = imdb_df.drop('genres', axis=1)

In [11]:
# directors_df = imdb_df['directors'].str.get_dummies(sep=',')
# directors_df = directors_df.add_prefix('dir_')
# imdb_df = pd.concat([imdb_df, directors_df], axis=1)
# imdb_df = imdb_df.drop('directors', axis=1)

In [13]:
actor_df = imdb_df['actor'].str.get_dummies(sep=',')
actor_df = actor_df.add_prefix('actor_')
imdb_df = pd.concat([imdb_df, actor_df], axis=1)
imdb_df = imdb_df.drop('actor', axis=1)

In [14]:
# define the bins
bins = [0, 2.4, 5.0, 7.4, 10]

# define the bin labels
labels = ['Terrible', 'Poor', 'Average', 'Excellent']

# create a new column with the bin labels
imdb_df['binRating'] = pd.cut(imdb_df['averageRating'], bins=bins, labels=labels, include_lowest=True)

imdb_df = imdb_df.drop('averageRating', axis=1)
imdb_df = pd.get_dummies(imdb_df, columns=['binRating'], prefix='bin')
imdb_df.head()

,runtimeMinutes,genre_Action,genre_Adventure,genre_Animation,genre_Biography,genre_Comedy,genre_Crime,genre_Documentary,genre_Drama,genre_Family,...,actor_nm9500254,actor_nm9503133,actor_nm9576872,actor_nm9858131,actor_nm9862858,actor_nm9877392,bin_Terrible,bin_Poor,bin_Average,bin_Excellent
tconst,,,,,,,,,,,,,,,,,,,,,
tt0035423,118,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
tt0118589,104,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
tt0118694,98,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
tt0120202,105,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
tt0120630,84,0,1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [15]:
from sklearn.preprocessing import StandardScaler

X = imdb_df.drop(['bin_Average', 'bin_Excellent', 'bin_Poor', 'bin_Terrible'], axis=1)
y = imdb_df[['bin_Excellent']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
from sklearn.multioutput import MultiOutputClassifier

log_reg = LogisticRegression(max_iter=1000000, solver='saga')
log_reg = MultiOutputClassifier(log_reg)
log_reg.fit(X_train, y_train)

score = log_reg.score(X_test, y_test)
print(f"Accuracy: {score}")

/Users/wileybattles/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/wileybattles/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mo

Accuracy: 0.7698744769874477
Coefficients: [[-0.02897584  0.35655808  0.2671942  ...  0.         -0.17589768
  -0.22264624]]
Coefficients: [[ 0.0428269   0.0088475  -0.34329828 ...  0.          0.1467392
   0.2306299 ]]
Coefficients: [[-0.03030032  0.36652126  0.56242969 ...  0.         -0.00799933
  -0.01660267]]
Coefficients: [[-0.01776921  0.21363698 -0.97003314 ...  0.         -0.00276996
  -0.00415386]]
